In [1]:
import sys
sys.path.append('../')
sys.path.append('.')
import recoengi
import recoengi.cf as cf

import pandas as pd
from scipy import sparse
import numpy as np
from sklearn import metrics
import logging
logging.basicConfig(level=logging.DEBUG, format="%(levelname)s %(asctime)s %(message)s")

In [2]:
dtf_ratings = pd.read_csv("sampledata/ml-latest-small/ratings.csv", usecols=["userId", "movieId", "rating"])
dtf_tmp = dtf_ratings.groupby(["movieId"]).agg({"rating": "count"}).reset_index(drop=False)
dtf_tmp = dtf_tmp.loc[dtf_tmp.rating > 3,:]
dtf_ratings = dtf_ratings.loc[dtf_ratings.movieId.isin(dtf_tmp.movieId),:]

dtf_tmp = pd.DataFrame({"movieId": dtf_ratings.movieId.unique()})
dtf_tmp = dtf_tmp.reset_index(drop = False)
dtf_tmp = dtf_tmp.rename({"index": "movieId_new"}, axis = 1)
dtf_ratings = pd.merge(dtf_ratings, dtf_tmp, on = ["movieId"], how = "left").drop(["movieId"], axis=1).rename({"movieId_new": "movieId"}, axis=1)
dtf_ratings.userId = dtf_ratings.userId-1
dtf_ratings.rating = (dtf_ratings.rating >= 3) + 0.0

tmp_bln_split = np.random.choice([True, False], size=dtf_ratings.shape[0], replace=True, p=[0.8, 0.2])
dtf_train = dtf_ratings.loc[tmp_bln_split, ["userId", "movieId", "rating"]]
dtf_test = dtf_ratings.loc[~tmp_bln_split, ["userId", "movieId", "rating"]]

M = sparse.csr_matrix((dtf_train.rating, (dtf_train.userId, dtf_train.movieId)))

In [3]:
cfm = cf.CFM(M)
cfm.computeEverything(bln_bin=False, bln_norm=True, flt_ths=0.01, ntop=64, flt_lb=-1)

DEBUG 2019-10-13 13:17:22,470 M matrix has shape 610x4180.
DEBUG 2019-10-13 13:17:22,476 B matrix has shape 610x4180.
DEBUG 2019-10-13 13:17:22,497 B matrix has sparsity 2.387951996234999%.
DEBUG 2019-10-13 13:17:22,499 Computing the similarity matrix S ...
DEBUG 2019-10-13 13:17:22,520 S matrix has shape 610x610.
DEBUG 2019-10-13 13:17:22,520 Computing the matrix SNORMALIZED ...
DEBUG 2019-10-13 13:17:22,522 Computing the matrix SCORES ...
DEBUG 2019-10-13 13:17:22,568 SCORES matrix has shape 610x4180.
DEBUG 2019-10-13 13:17:22,568 Computing the matrix AMOUNTS ...
DEBUG 2019-10-13 13:17:22,610 AMOUNTS matrix has shape 610x4180.
DEBUG 2019-10-13 13:17:22,610 Computing the performances ...
DEBUG 2019-10-13 13:17:22,739 Average global scores difference: 0.050420268478464286.
DEBUG 2019-10-13 13:17:22,739 Average positive scores difference: 0.7467839997832604.
WARNING 2019-10-13 13:17:22,740 The scores model could not perform better than a random model! Further tests needed!
DEBUG 2019-10

In [4]:
dtf_pred = pd.DataFrame(pd.Series(dict(cfm.SCORES.todok().items()))).reset_index(drop=False)
dtf_pred.columns = ["userId", "movieId", "predicted_score"]
dtf_train = pd.merge(dtf_train, dtf_pred, on=["userId", "movieId"], how="inner")
dtf_test = pd.merge(dtf_test, dtf_pred, on=["userId", "movieId"], how="inner")

In [5]:
fpr, tpr, thresholds = metrics.roc_curve(y_true=dtf_train.rating+1, y_score=dtf_train.predicted_score, pos_label=2)
print("AUC on training set: " + str(metrics.auc(fpr, tpr)) + ".")
fpr, tpr, thresholds = metrics.roc_curve(y_true=dtf_test.rating+1, y_score=dtf_test.predicted_score, pos_label=2)
print("AUC on test set: " + str(metrics.auc(fpr, tpr)) + ".")

AUC on training set: 0.6795735934614884.
AUC on test set: 0.6512761220040132.
